<a href="https://colab.research.google.com/github/IgorJoaquimn/2023-TSP-Annealing/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numba import jit
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
from scipy.spatial import ConvexHull,convex_hull_plot_2d
from random import sample
import time

In [ ]:
@jit(nopython=True)
def custo(N,path,dist):
    # calcula a distancia total percorrida pela caminhada
    ener = 0
    for i in range(N-1):
        ener += dist[path[i],path[i+1]]
    ener += dist[path[0],path[N-1]]     # conecta a última e a primeira cidades do caminho

    return ener

In [ ]:
@jit(nopython=True)
def newpath1(N,path):
    newpath = np.zeros(N,dtype=np.int16)
    i=np.random.randint(N)   # escolhe uma posição aleatória da caminhada
    j=i
    while j==i:
        j=np.random.randint(N)  # escolhe outra posição
    if i>j:                    # ordena os índices
        ini = j
        fin = i
    else:
        ini = i
        fin = j

    for k in range(N):        # inverte o sentido em que percorre o caminho entre os indices escolhidos
        if k >= ini and k <= fin:
            newpath[k] = path[fin-k+ini]
        else:
            newpath[k] = path[k]

    return newpath,ini,fin

@jit(nopython=True)
def newpath2(N,path):
    i = np.random.randint(N)   # escolhe uma posição aleatória da caminhada
    j = np.random.randint(N)

    if i>j:                    # ordena os índices
        ini = j
        fin = i
    else:
        ini = i
        fin = j

    path[ini:(fin+1)] = path[ini:(fin+1)][::-1]
    return path,ini,fin

def newpath3(N,path):
    i = np.random.randint(N-1)   # escolhe uma posição aleatória da caminhada
    j = ((i+N)**2 + 11*i) % (N-1)
    if i>j:                    # ordena os índices
        ini = j
        fin = i
    else:
        ini = i
        fin = j

    return np.concatenate([path[:ini+1],path[fin:],path[ini+1:fin]]),ini,fin

def newpath4(N,path):
  ini, fin = sorted(sample(range(0,N),2))
  np.concatenate([path[:ini+1],path[fin:],path[ini+1:fin]])
  return  np.concatenate([path[:ini+1],path[fin:],path[ini+1:fin]]), ini, fin

@jit(nopython=True)
def newpath5(N,path):
  i = np.random.randint(N)  # escolhe uma posição aleatória da caminhada
  j = i
  while j == i:
    j = np.random.randint(N)  # escolhe outra posição

  # Ordena os índices
  ini, fin = sorted([i, j])

  # Inverte o sentido em que percorre o caminho entre os índices escolhidos
  newpath = np.where(np.logical_and(np.arange(N) >= ini, np.arange(N) <= fin), path[fin - np.arange(ini)], path)

In [ ]:
def mcstep(N,beta,en,path,best_e,best_p,dist):
    # realiza um passo de Monte Carlo
    np1 = np.zeros(N,dtype=np.int16)
    np1,ini,fin = newpath1(N,path) # propoe um novo caminho


    # determina a diferença de energia
    esq = ini-1                 # cidade anterior a inicial
    if esq < 0: esq=N-1         # condicao de contorno
    dir = fin +1                # cidade apos a final
    if dir > N-1: dir=0         # condicao de contorno

    de = -dist[path[esq],path[ini]] - dist[path[dir],path[fin]]+ dist[np1[esq],np1[ini]] + dist[np1[dir],np1[fin]]

    if de < 0:         # aplica o criterio de Metropolis
        en += de
        path = np1
        if en < best_e:  # guarda o melhor caminho gerado até o momento
            best_e = en
            best_p = path
    else:              # aplica o criterio de Metropolis
        if np.random.random() < np.exp(-beta*de):
            en += de
            path = np1

    return en,path,best_e,best_p

In [ ]:
NPONTOS = 1000
X = np.random.uniform(0,1,NPONTOS)
Y = np.random.uniform(0,1,NPONTOS)

pontos = np.array([X,Y]).T
dist = cdist(pontos,pontos,metric="euclidean")
rv = multivariate_normal([0, 0.5],np.cov(pontos.T)).pdf(pontos)
path = np.argsort(rv)

In [ ]:
N = NPONTOS
beta = 1
en = custo(N,path,dist)
best_e = en
best_p = path
# é isso? sim

In [ ]:
N = NPONTOS
beta = 1
en = custo(N,path,dist)
best_e = en
best_p = path
# é isso? sim


t_0 = 1
alpha = 0.9

t_anterior = t_0
t_i         = alpha*t_anterior
limiar = np.exp(-100)

while((t_anterior - t_i)>limiar):
  t_anterior = t_i
  t_i         = alpha*t_anterior
  beta = 1/t_i
  for i in range(5):
    en,path,best_e,best_p = mcstep(N,beta,en,path,best_e,best_p,dist)
    en    = best_e
    path  = best_p

print(best_e)

222.8432211921134


In [ ]:
!jupyter nbconvert Relatorio_TSP_GiovanaAssis_IgorJoaquim.ipynb --to pdf


[NbConvertApp] Converting notebook Relatorio_TSP_GiovanaAssis_IgorJoaquim.ipynb to pdf
[NbConvertApp] Support files will be in Relatorio_TSP_GiovanaAssis_IgorJoaquim_files/
[NbConvertApp] Making directory ./Relatorio_TSP_GiovanaAssis_IgorJoaquim_files
[NbConvertApp] Making directory ./Relatorio_TSP_GiovanaAssis_IgorJoaquim_files
[NbConvertApp] Making directory ./Relatorio_TSP_GiovanaAssis_IgorJoaquim_files
[NbConvertApp] Making directory ./Relatorio_TSP_GiovanaAssis_IgorJoaquim_files
[NbConvertApp] Writing 50110 bytes to notebook.tex
[NbConvertApp] Building PDF
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 280, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3